### Load Data Input 

In [1]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
#print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)
prefix = 'NLP_AWS/NLP_AWS' #Replace with the prefix under which you want to store the data if needed

sagemaker-us-east-1-023375022819


In [3]:
!wget https://s3.amazonaws.com/fast-ai-nlp/amazon_review_polarity_csv.tgz

--2019-08-28 02:25:12--  https://s3.amazonaws.com/fast-ai-nlp/amazon_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.205.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.205.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 688339454 (656M) [application/x-tar]
Saving to: ‘amazon_review_polarity_csv.tgz.2’

amazon_review_polar 100%[===================>] 656.45M  52.5MB/s    in 14s     

2019-08-28 02:25:26 (46.9 MB/s) - ‘amazon_review_polarity_csv.tgz.2’ saved [688339454/688339454]



In [4]:
!tar -xzvf amazon_review_polarity_csv.tgz

amazon_review_polarity_csv/
amazon_review_polarity_csv/train.csv
amazon_review_polarity_csv/readme.txt
amazon_review_polarity_csv/test.csv


In [2]:
!head amazon_review_polarity_csv/train.csv -n 3

"2","Stuning even for the non-gamer","This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^"
"2","The best soundtrack ever to anything.","I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."
"2","Amazing!","This soundtrack is my favorite mu

#### Then upload file from sagemaker folder to S3 

In [10]:
%%time

train_channel = prefix + '/train'
test_channel = prefix + '/test'

sess.upload_data(path='amazon_review_polarity_csv/train.csv', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='amazon_review_polarity_csv/test.csv', bucket=bucket, key_prefix=test_channel)

CPU times: user 12.7 s, sys: 6.89 s, total: 19.6 s
Wall time: 18.9 s


### Read datafile as CSV

In [3]:
import pandas as pd

df = pd.read_csv("s3://sagemaker-us-east-1-023375022819/NLP_AWS/NLP_AWS/train/train.csv", names=["Label", "Title", "Review"])

In [4]:
neg_df = df[df['Label'] == 1]

In [5]:
neg_df.head()

,Label,Title,Review
6,1,Buyer beware,"This is a self-published book, and if you want..."
10,1,The Worst!,A complete waste of time. Typographical errors...
13,1,Oh please,I guess you have to be a romance novel lover f...
14,1,Awful beyond belief!,I feel I have to write to keep others from was...
15,1,Don't try to fool us with fake reviews.,It's glaringly obvious that all of the glowing...


In [7]:
neg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1800000 entries, 6 to 3599998
Data columns (total 3 columns):
Label     int64
Title     object
Review    object
dtypes: int64(1), object(2)
memory usage: 54.9+ MB


In [8]:
extract_df = neg_df.iloc[:,0:20000]['Review']

In [9]:
extract_df.head(10)

6     This is a self-published book, and if you want...
10    A complete waste of time. Typographical errors...
13    I guess you have to be a romance novel lover f...
14    I feel I have to write to keep others from was...
15    It's glaringly obvious that all of the glowing...
19    sizes are much smaller than what is recomended...
20    This model may be ok for sedentary types, but ...
22    Rather than scratches and insect droppings, th...
25    I have had the charger for more than two years...
26    I bought one of these chargers..the instructio...
Name: Review, dtype: object

In [ ]:
extract_df.to_csv('extract_df.csv', index=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


In [5]:
%%time

import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
#print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)
prefix = 'NLP_AWS/NLP_AWS' #Replace with the prefix under which you want to store the data if needed

comprehend_channel = prefix + '/comprehend'

sess.upload_data(path='extract_df.csv', bucket=bucket, key_prefix=comprehend_channel)

NoCredentialsError: Unable to locate credentials